# Homework 10 - Kubernetes Deployment

## Setup

First, clone the repo and build the Docker image in WSL:

```bash
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
cd machine-learning-zoomcamp/cohorts/2025/05-deployment/homework
docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .
```

## Question 1: Test Docker Image

Run in WSL terminal 1:
```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10
```

Run in WSL terminal 2:
```bash
python q6_test.py
```

In [ ]:
# q6_test.py content - create this file
q6_test_code = '''import requests

url = "http://localhost:9696/predict"

client = {
    "job": "management",
    "duration": 400,
    "poutcome": "success"
}

response = requests.post(url, json=client).json()
print(response)
'''

with open('q6_test.py', 'w') as f:
    f.write(q6_test_code)
print("q6_test.py created")

## Question 2: Kind Version

Run in WSL:
```bash
kind --version
```

## Create Cluster

```bash
kind create cluster
kubectl cluster-info
```

## Question 3: Smallest Deployable Unit

Answer: **Pod**

A Pod is the smallest deployable computing unit in Kubernetes.

## Question 4: Service Type

Run in WSL:
```bash
kubectl get services
```

The default `kubernetes` service has type **ClusterIP**.

## Question 5: Load Docker Image to Kind

Answer: **kind load docker-image**

Run in WSL:
```bash
kind load docker-image zoomcamp-model:3.13.10-hw10
```

## Question 6: Deployment Config

In [ ]:
deployment_yaml = '''apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.13.10-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: 9696
'''

with open('deployment.yaml', 'w') as f:
    f.write(deployment_yaml)
print("deployment.yaml created")
print("\nPort value: 9696")

Apply deployment in WSL:
```bash
kubectl apply -f deployment.yaml
kubectl get pods
```

## Question 7: Service Config

In [ ]:
service_yaml = '''apiVersion: v1
kind: Service
metadata:
  name: subscription
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696
'''

with open('service.yaml', 'w') as f:
    f.write(service_yaml)
print("service.yaml created")
print("\nAnswer for <???>: subscription")

Apply service in WSL:
```bash
kubectl apply -f service.yaml
kubectl get services
```

## Test the Service

Port forward in WSL:
```bash
kubectl port-forward service/subscription 9696:80
```

Then run q6_test.py in another terminal.

## Autoscaling Setup

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
kubectl get hpa
```

## Question 8: Load Test

In [ ]:
load_test_code = '''import requests
from time import sleep

url = "http://localhost:9696/predict"

client = {
    "job": "management",
    "duration": 400,
    "poutcome": "success"
}

while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
'''

with open('load_test.py', 'w') as f:
    f.write(load_test_code)
print("load_test.py created")

Run load test:
```bash
# Terminal 1: Watch HPA
kubectl get hpa subscription-hpa --watch

# Terminal 2: Run load test
python load_test.py
```

Max replicas should scale up to **3** under load.